In [6]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import init, MarginRankingLoss
from transformers import BertModel, RobertaModel
from transformers import BertTokenizer, RobertaTokenizer
from torch.optim import Adam
from distutils.version import LooseVersion
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torch.autograd import Variable
from transformers import AutoConfig, AutoModel, AutoTokenizer
import nltk
import re
import Levenshtein
import spacy
import en_core_web_sm
import torch.optim as optim
from torch.distributions import Categorical
from numpy import linalg as LA
from transformers import AutoModelForMaskedLM
from nltk.corpus import wordnet
import torch.nn.functional as F
import random
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import init, MarginRankingLoss
from transformers import BertModel, RobertaModel
from transformers import BertTokenizer, RobertaTokenizer
from torch.optim import Adam
from distutils.version import LooseVersion
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torch.autograd import Variable
from transformers import AutoConfig, AutoModel, AutoTokenizer
import nltk
import re
import Levenshtein
import spacy
import en_core_web_sm
import torch.optim as optim
from torch.distributions import Categorical
from numpy import linalg as LA
from transformers import AutoModelForMaskedLM
from nltk.corpus import wordnet
import torch.nn.functional as F
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_recall_fscore_support
from nltk.corpus import words as wal
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
import re
from bs4 import BeautifulSoup
import csv
# nltk.download("punkt")
# nltk.download("words")
# nltk.download('wordnet')

In [2]:
def freeze(model):
    for name, param in model.named_parameters():
        param.requires_grad = True
#         if name.startswith("model.roberta.encoder.layer.0"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.1"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.2"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.3"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.4"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.5"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.6"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.7"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.8"):
#             param.requires_grad = False
#         if name.startswith("model.roberta.encoder.layer.9"):
#             param.requires_grad = False
    return model

In [3]:
class MyDataset(Dataset):
    def __init__(self,file_name):
        df1 = pd.read_csv(file_name)
        df1 = df1.fillna("")
        res = df1['X']
#         ab = df1['X']
#         res = [sub.replace("<mask>", "[MASK]") for sub in ab]
        self.X_list = res
        self.y_list = df1['y']
        self.clean = df1['clean_text']
    def __len__(self):
        return len(self.X_list)
    def __getitem__(self,idx):
        mapi = []
        mapi.append(self.X_list[idx])
        mapi.append(self.y_list[idx])
        mapi.append(self.clean[idx])
        return mapi

In [4]:
class Step1_model(nn.Module):
    def __init__(self, hidden_size=512):
        self.old_mhs = []
        self.old_log_probs = []
        self.old_logits = []
        super(Step1_model, self).__init__()
        self.hidden_size = hidden_size
#         self.model = AutoModel.from_pretrained("roberta-base")
#         self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
#         self.config = AutoConfig.from_pretrained("roberta-base")
        self.model = AutoModelForMaskedLM.from_pretrained('microsoft/graphcodebert-base')
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
        self.config = AutoConfig.from_pretrained("microsoft/graphcodebert-base")
        self.linear_layer = nn.Linear(self.model.config.vocab_size, self.model.config.vocab_size)

#         self.model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
#         self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#         self.config = AutoConfig.from_pretrained("bert-base-cased")
        for param in self.model.base_model.parameters():
            param.requires_grad = True
    def foo (self,data):
        result = []
        if type(data) == tuple:
            return data[1]
        if type(data) == list:
            for inner in data:
                result.append(foo(inner))
        res = []
        for a in result[0]:
            res.append(a[:2])
        return res
    def loss_func1(self, word, y):
        if word =='NA':
            return torch.full((1,), fill_value=100)
        try:
            pred_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
            target_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
            pred_tag = self.foo(nltk.pos_tag(pred_list))
            target_tag = self.foo(nltk.pos_tag(target_list))
            str1 = ' '.join(pred_tag)  # Convert lists to strings
            str2 = ' '.join(target_tag)
            distance = Levenshtein.distance(str1, str2)
            dist = torch.Tensor([distance])
        except:
            dist = torch.Tensor([2*len(target_list)])
        return dist
    def loss_func2(self, word, y):
        if word =='NA':
            return  torch.full((1,), fill_value=100)
        nlp = en_core_web_sm.load()
        pred_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
        target_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
        try:
            str1 = ' '.join(pred_list)  # Convert lists to strings
            str2 = ' '.join(target_list)
            tokens1 = nlp(str1)
            tokens2 = nlp(str2)
            # Calculate the average word embedding for each string
            embedding1 = sum(token.vector for token in tokens1) / len(tokens1)
            embedding2 = sum(token.vector for token in tokens2) / len(tokens2)
            # Calculate the cosine similarity between the embeddings
            w1= LA.norm(embedding1)
            w2= LA.norm(embedding2)
            distance = 1 - (embedding1.dot(embedding2) / (w1 * w2))
            dist = torch.Tensor([distance])
        except:
            dist = torch.Tensor([1])
        return dist
    def compute_loss(self, logits, target_word):
        # Apply softmax to obtain probabilities
        probabilities = F.softmax(logits, dim=-1)
        log_probs = torch.log(probabilities)
        target_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', target_word)
        joined_sentence = " ".join(target_list)
        target_tokens = self.tokenizer.tokenize(joined_sentence)
        loss = 0
        for j in range(len(target_tokens)):
            target_index = self.tokenizer.convert_tokens_to_ids(target_tokens[j])
            # Retrieve the probability of the target word
            target_prob = probabilities[:, target_index]  # Assuming target_index is known
            # Compute the negative log-likelihood loss
            l = -torch.log(target_prob)
            loss+=l
        return {'loss':loss,'log_probs':log_probs}
    def forward(self, mapi):
        english_dict = set(wal.words())
        X_init = mapi[0]
        y = mapi[1]
        print(y)
        nl = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
        lb = ' '.join(nl).lower()
        x = self.tokenizer.tokenize(lb)
        num_sub_tokens_label = len(x)
        X_init = X_init.replace("[MASK]", " ".join([self.tokenizer.mask_token] * num_sub_tokens_label))
        preds = []
        for m in range(num_sub_tokens_label):
            print(m)
            tokens = self.tokenizer.encode_plus(X_init, add_special_tokens=False,return_tensors='pt')
            input_id_chunki = tokens['input_ids'][0].split(510)
            input_id_chunks = []
            mask_chunks  = []
            mask_chunki = tokens['attention_mask'][0].split(510)
            for tensor in input_id_chunki:
                input_id_chunks.append(tensor)
            for tensor in mask_chunki:
                mask_chunks.append(tensor)
            xi = torch.full((1,), fill_value=101)
            yi = torch.full((1,), fill_value=1)
            zi = torch.full((1,), fill_value=102)
            for r in range(len(input_id_chunks)):
                input_id_chunks[r] = torch.cat([xi, input_id_chunks[r]],dim = -1)
                input_id_chunks[r] = torch.cat([input_id_chunks[r],zi],dim=-1)
                mask_chunks[r] = torch.cat([yi, mask_chunks[r]],dim=-1)
                mask_chunks[r] = torch.cat([mask_chunks[r],yi],dim=-1)
            di = torch.full((1,), fill_value=0)
            for i in range(len(input_id_chunks)):
                # get required padding length
                pad_len = 512 - input_id_chunks[i].shape[0]
                # check if tensor length satisfies required chunk size
                if pad_len > 0:
                    # if padding length is more than 0, we must add padding
                    for p in range(pad_len):
                        input_id_chunks[i] = torch.cat([input_id_chunks[i],di],dim=-1)
                        mask_chunks[i] = torch.cat([mask_chunks[i],di],dim=-1)
            input_ids = torch.stack(input_id_chunks)
            attention_mask = torch.stack(mask_chunks)
            input_dict = {
                'input_ids': input_ids.long(),
                'attention_mask': attention_mask.int()
            }
            maski = []
            u = 0
            ad = 0
            for l in range(len(input_dict['input_ids'])):
                masked_pos = []
                for i in range(len(input_dict['input_ids'][l])):
                    if input_dict['input_ids'][l][i] == 50264: #103
                        u+=1
                        if i != 0 and input_dict['input_ids'][l][i-1] == 50264:
                            continue
                        masked_pos.append(i)
                        ad+=1
                maski.append(masked_pos)
#             if u<8:
#                 print("maski: ", maski)
            print('number of mask tok',u)
            print('number of seq', ad)
            with torch.no_grad():
                output = self.model(**input_dict)
            last_hidden_state = output[0].squeeze()
            l_o_l_sa = []
            lhs_agg = []
            if len(maski) == 1:
                masked_pos = maski[0]
                lhs_agg.append(last_hidden_state)
                for k in masked_pos:
                    l_o_l_sa.append(last_hidden_state[k])
            else:
                for p in range(len(maski)):
                    lhs_agg.append(last_hidden_state[p])
                    masked_pos = maski[p]
                    for k in masked_pos:
                        l_o_l_sa.append(last_hidden_state[p][k])
            sum_state = l_o_l_sa[0]
            lhs = lhs_agg[0]
            for i in range(len(lhs_agg)):
                if i == 0:
                    continue
                lhs+=lhs_agg[i]
            lhs/=len(lhs_agg)
            for i in range(len(l_o_l_sa)):
                if i == 0:
                    continue
                sum_state += l_o_l_sa[i]
            yip = len(l_o_l_sa)
            sum_state /= yip
    #         try:
            idx = torch.topk(sum_state, k=1, dim=0)[1]
            qw = [self.tokenizer.decode(i.item()).strip() for i in idx][0]
            preds.append(qw)
            xl = X_init.split()
            xxl = []
            for p in range(len(xl)):
                if xl[p] == self.tokenizer.mask_token:
                    if p != 0 and xl[p-1] == self.tokenizer.mask_token:
                        xxl.append(xl[p])
                        continue
                    xxl.append(qw)
                    continue
                xxl.append(xl[p])
            X_init = " ".join(xxl)
        we = preds[0]
        for t in range(len(preds)):
            if t == 0:
                continue
            we+=preds[t].capitalize()

        word_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', we)
        z = 0
        while z < len(word_list) - 1:
            word1 = word_list[z].lower()
            word2 = word_list[z + 1].lower()
            merged_word = word1 + word2

            if word1 not in english_dict and word2 not in english_dict:
                # Merge the 2 words and insert the resulting word at index (z)
                word_list[z] = merged_word
                word_list.pop(z + 1)

            elif word1 in english_dict and word2 not in english_dict:
                # Combine the words to see if the resulting word is in the dictionary
                if merged_word in english_dict:
                    # Merge the words and insert the merged word at index (z)
                    word_list[z] = merged_word
                    word_list.pop(z + 1)
                else:
                    if not (z+2)<len(word_list):
                        z+=1
                        continue
                    a = merged_word+word_list[z + 2].lower()
                    if a in english_dict:
                        word_list[z] = a
                        word_list.pop(z + 1)
                        word_list.pop(z + 2)
                    else:
                        z+=1
                        continue
            elif word1 not in english_dict and word2 in english_dict:
                # Combine the words to see if the resulting word is in the dictionary
                if merged_word in english_dict:
                    # Merge the words and insert the merged word at index (z)
                    word_list[z] = merged_word
                    word_list.pop(z + 1)
                else:
                    if not (z+2)<len(word_list):
                        z+=1
                        continue
                    a = merged_word+word_list[z + 2].lower()
                    if a in english_dict:
                        word_list[z] = a
                        word_list.pop(z + 1)
                        word_list.pop(z + 2)
                    else:
                        z+=1
                        continue
            else:
                z += 1
                continue
            z+=1
        fin_str = ""
        for o in range(len(word_list)):
            if o == 0:
                fin_str+=word_list[o].lower()
                continue
            fin_str+=word_list[o].lower().capitalize()
        word = fin_str
#         except:
#             word = "NA"         
        pred_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
        target_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
        target_string = ' '.join(pred_list)
        predicted_string = ' '.join(target_list)
        precision, recall, f1_score, _ = precision_recall_fscore_support([target_string], [predicted_string], average='micro')
        rank = 0
        for d, wordh in enumerate(pred_list):
            if d >= len(target_list):
                break
            if wordh == target_list[d]:
                rank = d + 1
                break
        mrr = 1.0 / rank if rank > 0 else 0.0
        target_set = set(target_list)
        predicted_set = set(pred_list)

        # Calculate Precision at K for K=3 (top 3 predictions)
        K = 3
        top_k_predictions = pred_list[:K]

        # Count the number of correct predictions in the top K
        correct_predictions = sum(1 for word in top_k_predictions if word in target_set)

        # Calculate Precision at K
        pak = correct_predictions / K
        print ("Guess : ",word)
#         maxi = Variable(torch.tensor(0.5, dtype=torch.float), requires_grad = True)
        maxi = Variable(0.2*self.loss_func2(word,y) + 0.8*self.loss_func1(word,y), requires_grad = True)
#         maxi.requires_grad()
        
        
        logits = self.linear_layer(lhs)
        cl = self.compute_loss(logits,y)
        if len(self.old_mhs) == 0:
            print("one")
            old_log_probs = torch.rand_like(cl['log_probs'])
            logits1 = torch.rand_like(logits)
            self.old_mhs.append(sum_state)
            self.old_log_probs.append(cl['log_probs'])
            self.old_logits.append(logits)
            if last_hidden_state[0].shape[0] < 1000:
                hits = torch.rand_like(last_hidden_state[0][0])
            else:
                hits = torch.rand_like(last_hidden_state[0])
        else:
            old_log_probs = self.old_log_probs[-1]
            logits1 = self.old_logits[-1]
            hits = self.old_mhs[-1]
            if len(self.old_mhs) == 1:
                self.old_mhs.clear()
            if len(self.old_logits) == 1:
                self.old_logits.clear()
            if len(self.old_log_probs) == 1:
                self.old_log_probs.clear()
            self.old_mhs.append(sum_state)
            self.old_log_probs.append(cl['log_probs'])
            self.old_logits.append(logits)
            
        return {'pak':pak,'mrr':mrr,'f1':f1_score,'returned_word':word, 'mhs':sum_state,'old_mhs':hits,'old_logits':logits1, 'actual_pred':word, 'loss':maxi, 'compute_loss':cl['loss'],'log_probs':cl['log_probs'],'logits':logits, 'old_log_probs':old_log_probs}

In [5]:
def extract_comments(java_code):
    pattern = r"(?<!\S)(\/\/[^\r\n]*|\/\*(?:(?!\*\/).|[\r\n])*?\*\/)"
    comments = re.findall(pattern, java_code)
#     print(java_code)
    mask_indices = [m.start() for m in re.finditer(r"[MASK]", java_code)]
    pt = [m.start() for m in re.finditer(pattern, java_code)]
    if len(mask_indices) == 0:
        return []
    fir = mask_indices[0]
    las = mask_indices[-1]
    keep = []
    for i in range(len(pt)):
        if pt[i]>=fir and pt[i]<=las:
            keep.append(i)
    try:
        if (keep[0] - 1) >=0:
            keep.append(keep[0] - 1)
    except:
        pass
    try:
        if (len(pt) - 1) >= keep[-1]:
            keep.append(keep[-1]+1)
    except:
        pass
    ret = []
    if 0 not in keep:
        if len(comments) != 0:
            ret.append(comments[0])
    for k in keep:
        if len(comments) > k:
            ret.append(comments[k])
    return ret

In [6]:
def calculate_co_occurrence_probability(str1, word2):
    # use graphCodeBert or Bert
    model = AutoModelForMaskedLM.from_pretrained('microsoft/graphcodebert-base')
    tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
    config = AutoConfig.from_pretrained("microsoft/graphcodebert-base")

    # Tokenize the sentence with the two words to create input tensors
    x = tokenizer.tokenize(word2)
    num_sub_tokens_label = len(x)
    sentence = str1
    for j in range(num_sub_tokens_label):
        sentence = sentence + " <mask>"
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,return_tensors='pt')
    masked_pos = []
    for j in range(len(inputs['input_ids'][0])):
        if inputs['input_ids'][0][j] == 50264: #103
            masked_pos.append(j)
    # Get the masked token position

    # Generate predictions for the masked token
    with torch.no_grad():
        predictions = model(**inputs).logits

    a = tokenizer.convert_tokens_to_ids(x)

    # Retrieve the probability of the second word being the correct prediction
    # try:
    word2_probability = 1
    for k in range(len(masked_pos)):
        c = torch.softmax(predictions[0, masked_pos[k]], dim=0)[a[k]]
        word2_probability *= c
    return word2_probability.item()

In [7]:
def find_word_meanings(word):
    synsets = wordnet.synsets(word) 
    if synsets:
        meanings = [synset.definition() for synset in synsets]
        return meanings
    else:
        return []

In [8]:
def extract_doc(java_code):
    lines = java_code.splitlines()
    lines_with_mask = [line for line in lines if "[MASK]" in line]
    return lines_with_mask

In [9]:
def rew(inp, word, clean_text):
    reward = torch.full((1,), fill_value=0.0)
    w_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
    xi = torch.full((1,), fill_value=1)
    yi = torch.full((1,), fill_value=2)
    for p in range(len(w_list)):
        w = w_list[p]
        if p==0:
            if not w.islower():
                reward-=yi
            continue
        if not w[0].isupper():
            reward-=xi
            w[0].lower()
        if not w.islower():
            reward-=yi
    comments = extract_comments(inp)
    str1 = ""
    for comment in comments:
        str1+=comment
        str1+="\n"
    pred_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
    str2 = ' '.join(pred_list)
    nlp = en_core_web_sm.load()
    tokens1 = nlp(str1)
    tokens2 = nlp(str2)
    if len(tokens1) == 0 or len(tokens2) == 0:
        reward +=0
    else:
        embedding1 = sum(token.vector for token in tokens1) / len(tokens1)
        embedding2 = sum(token.vector for token in tokens2) / len(tokens2)
        w1= LA.norm(embedding1)
        w2= LA.norm(embedding2)
        sim = (embedding1.dot(embedding2) / (w1 * w2))
        sim*=10
        simi = torch.full((1,), fill_value=sim)
        reward+=simi
    str1 = clean_text
    nlp = en_core_web_sm.load()
    tokens1 = nlp(str1)
    tokens2 = nlp(str2)
    if len(tokens1) == 0 or len(tokens2) == 0:
        reward +=0
    else:
        embedding1 = sum(token.vector for token in tokens1) / len(tokens1)
        embedding2 = sum(token.vector for token in tokens2) / len(tokens2)
        w1= LA.norm(embedding1)
        w2= LA.norm(embedding2)
        sim = (embedding1.dot(embedding2) / (w1 * w2))
        sim*=10
        simi = torch.full((1,), fill_value=sim)
        reward+=simi
    w_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
    o = 0
    tot_prob = 1
    if len(w_list) > 1:
        str1 = ""
        for wordi in w_list:
            if o == 0:
                str1+=wordi
                str1+=" "
                o+=1
                continue
            tot_prob = tot_prob*calculate_co_occurrence_probability(str1,wordi)
            str1+=wordi
            str1+=" "
    a = len(w_list)
    tot_prob*=10**(2**a)
    tot_prob = np.log(tot_prob)
    di = torch.full((1,), fill_value=tot_prob)
    reward+= di
    w_list = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', word)
    counter = 0
    po = ['NN', 'FW', 'NNS']
    for l in range(len(w_list)):
        b = nltk.pos_tag([w_list[l]])
        if b[0][1] in po:
            a = find_word_meanings(w_list[l])
            counter+= len(a)
    counter-=len(w_list)
    zi = torch.full((1,), fill_value=counter)
    reward-=zi
    return reward

In [10]:
def reward_function(inputs,model_output):
    for i in range(len(model_output)):
        global u
        val = rew(inputs[0][i],model_output[i]['actual_pred'], inputs[2][i])
        if u == 0:
            rewards = val
            u+=1
        else:
            xi = val
            rewards = torch.cat((rewards, xi), dim=0)
    return rewards

In [11]:
# with current batch_size of 5
u = 0
def ppo_algorithm(transformer_model, reward_function, dataset, batch_size=1, clip_param=0.2,
                  value_loss_coef=0.5, entropy_coef=0.01, lr=0.001, kl_coef=0.1):
    optimizer = optim.Adam(transformer_model.parameters(), lr=lr)
    value_criterion = nn.MSELoss()
    running_loss = 0.
    last_loss = 0.
    global u
    global human_input_prob
    global dfn
    global epoch_number
    
    for batch in dataset:
        # Extract inputs and labels from the batch
        u = 0
        inputs = batch
#         print(inputs)
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        # Make predictions for this batch
        model_output = []
        output = []
#         try:
        for i in range(len(inputs[0])):
            l = []
            l.append(inputs[0][i])
            l.append(inputs[1][i])
            opi = transformer_model(l)
            model_output.append(opi)
            output.append(opi['actual_pred'])
#         except:
#             continue
        # Calculate rewards
        rewardsi = reward_function(inputs, model_output)
        # Calculate value loss
        for i in range(len(model_output)):
            if i == 0:
                values = model_output[i]['logits']
                old_values =  model_output[i]['old_logits']
                mhs = model_output[i]['mhs']
                old_mhs = model_output[i]['old_mhs']
                log_probs = model_output[i]['log_probs']
                old_log_probs = model_output[i]['old_log_probs']
            elif i == 1:
                xj = model_output[i]['logits']
                yj = model_output[i]['old_logits']
                zj = model_output[i]['mhs']
                dj = model_output[i]['old_mhs']
                aj = model_output[i]['log_probs']
                qj = model_output[i]['old_log_probs']
                values = torch.stack((values, xj), dim=0)
                old_values = torch.stack((old_values, yj), dim=0)
                mhs = torch.stack((mhs, zj), dim=0)
                old_mhs = torch.stack((old_mhs, dj), dim=0)
                log_probs = torch.stack((log_probs, aj), dim=0)
                old_log_probs = torch.stack((old_log_probs, qj), dim=0)
            else:
                xj = model_output[i]['logits']
                xj = xj.unsqueeze(0)
                xj = xj.expand(1,-1,-1)
                yj = model_output[i]['old_logits']
                yj = yj.unsqueeze(0)
                yj = yj.expand(1,-1,-1)
                zj = model_output[i]['mhs']
                zj = zj.unsqueeze(0)
                zj = zj.expand(1,-1)
                dj = model_output[i]['old_mhs']
                dj = dj.unsqueeze(0)
                dj = dj.expand(1,-1)
                aj = model_output[i]['log_probs']
                aj = aj.unsqueeze(0)
                aj = aj.expand(1,-1,-1)
                qj = model_output[i]['old_log_probs']
                qj = qj.unsqueeze(0)
                qj = qj.expand(1,-1,-1)
                values = torch.cat((values, xj), dim=0)
                old_values = torch.cat((old_values, yj), dim=0)
                mhs = torch.cat((mhs, zj), dim=0)
                old_mhs = torch.cat((old_mhs, dj), dim=0)
                log_probs = torch.cat((log_probs, aj), dim=0)
                old_log_probs = torch.cat((old_log_probs, qj), dim=0)
        vj = []
        cj = []
        for j in range(values.size()[0]):
            vj.append(value_criterion(old_values[j],values[j]))
            cj.append(value_criterion(old_mhs[j],mhs[j]).exp())
#         print(vj)
        ratios = torch.stack(cj, dim=0)
        value_loss = torch.stack(vj, dim=0)
        entropy_loss = -(log_probs * log_probs.exp()).sum(dim=-1).mean()
        h = random.random()
        kl_divergence = (old_log_probs.exp() * (old_log_probs - log_probs)).mean()
        if epoch_number < 2:
            h = 2
        if h < human_input_prob:
            t['inputs'].append(list(inputs[0]))
            t['y'].append(list(inputs[1]))
            t['output'].append(output)
            t['calc_reward'].append(rewardsi)
            t['entropy_loss'].append(entropy_loss)
            t['value_loss'].append(value_loss)
            t['ratios'].append(ratios)
            t['kl'].append(kl_divergence)
#             tdf = pd.DataFrame(t)
#             dfn = pd.concat([dfn,tdf])
            continue
        baseline = rewardsi.detach().mean()
        advantages = rewardsi - baseline
        value_loss *= advantages
        # Calculate action loss
        surrogate_loss = torch.min(ratios * advantages, torch.clamp(ratios, 1 - clip_param, 1 + clip_param) * advantages)
        action_loss = -surrogate_loss.mean()
        
        # Calculate total loss
        total_loss = value_loss_coef * value_loss + action_loss - entropy_coef * entropy_loss + kl_coef * kl_divergence
        # Backward pass and optimization step
        print("hurt")
        optimizer.zero_grad()
        total_loss.sum().backward()
        optimizer.step()
        running_loss += total_loss.item()
        if i % 1 == 0:
            last_loss = running_loss / 5 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
#             tb_writer.add_scalar('Loss1/train1', last_loss, tb_x)
            running_loss = 0.
    return transformer_model

In [12]:
# with current batch_size of 5
def human_ppo(transformer_model, clip_param=0.2, value_loss_coef=0.5, entropy_coef=0.01, lr=0.001, kl_coef = 0.1):
    optimizer = optim.Adam(transformer_model.parameters(), lr=lr)
    value_criterion = nn.MSELoss()
    running_loss = 0.
    last_loss = 0.
    global t
    global epoch_number
    global run_int
    m1 =  Step1_model()
    m1.load_state_dict(torch.load('var_runs/model_{}_{}'.format(run_int,epoch_number-1)))
    m2 =  Step1_model()
    m2.load_state_dict(torch.load('var_runs/model_{}_{}'.format(run_int,epoch_number-2)))
    for r in range(len(t['inputs'])):
        print("b")
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        # Make predictions for this batch
        inputs = t['inputs'][r]
        y_list = t['y'][r]
        entropy_loss = t['entropy_loss'][r]
        ratios = t['ratios'][r]
        value_loss = t['value_loss'][r]
        rewardsi = t['calc_reward'][r]
        output = t['output'][r]
        kl_divergence = t['kl'][r]
#         inputs = dfn.iloc[r]['examples']
#         y_list = dfn.iloc[r]['y']
#         entropy_loss = dfn.iloc[r]['entropy_loss']
#         value_loss = dfn.iloc[r]['value_loss']
#         ratios = dfn.iloc[r]['ratios']
#         output = dfn.iloc[r]['output']
#         rewards = dfn.iloc[r]['calc_rew']
        for q in range(len(inputs)):
            ans_list = []
            ans_list.append(output[q])
            l = []
            l.append(inputs[q])
            l.append(y_list[q])
            o = m1(l)
            o1 = m2(l)
            ans_list.append(o['actual_pred'])
            ans_list.append(o1['actual_pred'])
            print("Here is the code with masked variable name:")
            print(inputs[q])
            print("Which of the answers is better?")
            print("A. "+str(ans_list[0]))
            print("B. "+str(ans_list[1]))
            print("C. "+str(ans_list[2]))
            print("D. "+str(y_list[q]))
            ans = input("Type a, b, c, or d")
            if ans.lower() == 'a':
                rewardsi[q] += 200
            elif ans.lower() == 'b':
                rewardsi[q] -= 100
            elif ans.lower() == 'c':
                rewardsi[q] -= 200
            elif ans.lower() == 'd':
                rewardsi[q] -= 400
            else:
                print("error, no choice made")
        baseline = rewardsi.detach().mean()
        advantages = rewardsi - baseline
        value_loss *= advantages
        # Calculate action loss
        surrogate_loss = torch.min(ratios * advantages, torch.clamp(ratios, 1 - clip_param, 1 + clip_param) * advantages)
        action_loss = -surrogate_loss.mean()

        # Calculate entropy loss

        # Calculate total loss
        total_loss = value_loss_coef * value_loss + action_loss - entropy_coef * entropy_loss + kl_coef * kl_divergence
        # Backward pass and optimization step
        optimizer.zero_grad()
        total_loss.sum().backward()
        optimizer.step()
        running_loss += total_loss.item()
#         if r % 1 == 0:
#             last_loss = running_loss / 32 # loss per batch
#             print('  batch {} loss: {}'.format(i + 1, last_loss))
#             tb_x = epoch_index * len(train_loader) + i + 1
# #             tb_writer.add_scalar('Loss1/train1', last_loss, tb_x)
#             running_loss = 0.
    return transformer_model

In [ ]:
human_input_prob = 0.02
epoch_number = 0
EPOCHS = 10
run_int = 0
model = Step1_model()
# model = torch.load()
model = freeze(model)
optimizer = Adam(model.parameters(), lr=0.001)
myDs=MyDataset('dat.csv')
t = {'inputs':[],'y':[],'output':[],'calc_reward':[], 'ratios':[],
                'entropy_loss':[],'value_loss':[], 'kl':[]}
train_loader=DataLoader(myDs,batch_size=2,shuffle=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    model = ppo_algorithm(model, reward_function, train_loader)
    if epoch >= 2:
        model = human_ppo(model)
    for key in t.keys():
        t[key].clear()
    model_path = 'rl_runs/model_{}_{}'.format(run_int, epoch_number)
    torch.save(model.state_dict(), model_path)
    epoch_number += 1

Token indices sequence length is longer than the specified maximum sequence length for this model (10878 > 512). Running this sequence through the model will result in indexing errors


EPOCH 1:
yTrans

0
number of mask tok 8
number of seq 4
1
number of mask tok 4
number of seq 4
Guess :  yTrans
one
missed

0
number of mask tok 8
number of seq 4
1
number of mask tok 4
number of seq 4
Guess :  active
hurt


In [ ]:
#need to get tokenizer and model for this, generate summary of lines with <mask>,
# and for lines between first and last <mask>